# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv ../../02_activities/assignments/.env

In [2]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

c:\Users\adith\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
import numpy as np
from glob import glob

# Write your code below.

PRICE_DATA = os.getenv("PRICE_DATA")
all_parquet_files = glob(os.path.join(PRICE_DATA, "*/*/*.parquet"))
all_parquet_files [:10] # restricted file count to speed up processing of code
dd_px = dd.read_parquet(all_parquet_files).set_index("ticker")

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
dd_feat = (dd_px.groupby('ticker', group_keys=False).
           apply(lambda x: x.assign(Close_lag = x['Close'].shift(1))).
           assign(returns = lambda x: x['Close']/x['Close_lag'] - 1).
           assign(hi_lo_range = lambda x: x['High'] - x['Low']))


C:\Users\adith\AppData\Local\Temp\ipykernel_18792\2327342673.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [8]:
pandas_df = dd_feat.compute()
pandas_df['rolling_avg_return'] = pandas_df['returns'].rolling(window=10).mean()
print(pandas_df.tail()) # inspect final five rows

             Date       Open       High        Low      Close  Adj Close  \
ticker                                                                     
WRB    2015-12-24  24.035557  24.702223  24.004444  24.608889  20.422520   
WRB    2015-12-28  24.577778  24.653334  24.417778  24.635555  20.444639   
WRB    2015-12-29  24.733334  24.955557  24.608889  24.937778  20.695454   
WRB    2015-12-30  24.848888  25.026667  24.768888  24.808889  20.588486   
WRB    2015-12-31  24.719999  24.719999  24.333332  24.333332  20.193830   

         Volume      sector                      subsector  year  Close_lag  \
ticker                                                                        
WRB     1115550  Financials  Property & Casualty Insurance  2015  23.871111   
WRB     1121625  Financials  Property & Casualty Insurance  2015  24.608889   
WRB      802800  Financials  Property & Casualty Insurance  2015  24.635555   
WRB      604125  Financials  Property & Casualty Insurance  2015  24.937

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

Not necessary. Ok to do on Dask.


+ Would it have been better to do it in Dask? Why?

Better to do in dask as we are working with large dataset. Rolling operations are similar in Dask as to those in Pandas but designed to handle larger-than-memory datasets by breaking them into smaller computations that can be parallelized.

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.